# Tech Talent Dashboard Data Preprocessing

This notebook steps through the process of automating the cleaning and aggregation of raw data provided by the Bureau of Labor Statistics.  It contains the scripts represented by the **ETL Scripts** step of the architecture diagram below.

<img src='architecture_p1.png'>

# Key Assumptions

1. **Google Drive** is used to store the original, unmodified data files collected from various public and proprietary sources.  This location is referred to as `data` or `DATA_DIR` in various places throughout the project.
2. **Google Sheets** is used as a psuedo-database where processed data are stored individually within their own files.  This location is referred to as `db` or `DB_DIR` throughout the project.
3. **Tableau Public** can connect directly to data residing on Google Drive and/or Sheets.  However, it can only refresh automatically from Google Sheets data sources.  Automatic refreshes occur once a day, or on demand by the dashboard publisher.
4. **GitHub.com** hosts all of the python scripts and other necessary files required for this project.  The raw data files **are not** saved within the repository. All raw data files will be automatically downloaded from Google Drive when the scripts are executed. 
5. An authorized **API Key** must first be obtained from Google and placed into the project's root directory with the name `key.json`.

# High-level Overview

1. Connect to Google Drive and identify all raw data files contained in the project's `/data/` folders.
2. Download all raw data files to the project's local `/data/` folder.
3. Load and prepare the SOC code crosswalks.  These files are used to convert all SOC codes to their most recent versions.  Different (and sometimes multiple) crosswalks are required based on the release year of the data.
4. Load the MSA lookup list from `/data/lookups/lk_msa.xlsx`.  **This file determines which metros are included in the final dataset and dashboard.  Adding or removing MSAs from this list and re-running the scripts will add/remove them from the final dataset that feeds into the dashboard.**

| area  | area_title                               | state | peer_type         |
|-------|------------------------------------------|-------|-------------------|
| 28940 | Knoxville, TN                            | TN    | National Peer MSA |
| 24860 | Greenville-Anderson-Mauldin, SC          | SC    | National Peer MSA |
| 46140 | Tulsa, OK                                | OK    | National Peer MSA |
| 30780 | Little Rock-North Little Rock-Conway, AR | AR    | National Peer MSA |
| 17900 | Columbia, SC                             | SC    | National Peer MSA |
| 24660 | Greensboro-High Point, NC                | NC    | National Peer MSA |
| 12940 | Baton Rouge, LA                          | LA    | National Peer MSA |
| 31140 | Louisville/Jefferson County, KY-IN       | KY    | National Peer MSA |
| 13820 | Birmingham-Hoover, AL                    | AL    | Birmingham MSA    |
| 12220 | Auburn-Opelika, AL                       | AL    | Alabama Peer MSA  |
| 33660 | Mobile, AL                               | AL    | Alabama Peer MSA  |
| 33860 | Montgomery, AL                           | AL    | Alabama Peer MSA  |
| 46220 | Tuscaloosa, AL                           | AL    | Alabama Peer MSA  |

5. Clean each of the BLS and IPEDS files, applying crosswalks and making all necessary changes.
6. Exclude all data for metros not included in the list above to work around table size limits.
7. Save the resulting final data files into the `db` folder.
8. Connect to Google Sheets and overwrite all contents with the new data.

*Note: Tableau Public data connections to Google Sheets uses the file's unique id rather than the filename. Uploading a new copy of the file will also generate a new id for that file, which in turn will break the connection between the dashboard and the data.*

# Raw data files go in `data` folder 

Raw data files should be placed into their corresponding files within the ```\data\``` folder in Google Drive:

```
Google Drive
    ├─mcdc-tech-talent  
            ├─data
                ├─ bg
                ├─ bls_msa
                │    ├─ MSA_M20XX_dl.xlsx
                ├─ bls_national
                │    ├─ national_M20XX_dl.xlsx
                ├─ ipeds
                │    ├─ C20XX_A.zip
                │    ├─ HD20XX.zip
                ├─ lookups
                     ├─ CIP2020_SOC2018_Crosswalk.xlsx
                     ├─ Crosswalk2010to2020.csv
                     ├─ lk_awlevel.xlsx
                     ├─ lk_msa.xlsx
                     ├─ lk_sectors.xlsx
                     ├─ oes_2019_hybrid_structure.xlsx

```

1. `bg` folder is not needed, but can be used to store burning glass exports
2. `bls_msa` folder contains Metropolitan XLS files from https://www.bls.gov/oes/#data
3. `bls_national` folder contains National XLS files from https://www.bls.gov/oes/#data
4. `ipeds` folder contains Completions and Institutional Characteristics files from https://nces.ed.gov/ipeds/datacenter/DataFiles.aspx?goToReportId=7
5. `lookups` folder contains user created files prefixed with `lk_` and:
    - CIP2020_SOC2018_Crosswalk.xlsx from https://nces.ed.gov/ipeds/cipcode/Files/CIP2020_SOC2018_Crosswalk.xlsx
    - Crosswalk2010to2020.csv from https://nces.ed.gov/ipeds/cipcode/resources.aspx?y=56
    - oes_2019_hybrid_structure.xlsx from https://www.bls.gov/oes/oes_2019_hybrid_structure.xlsx
    

# The `db` folder will be automatically populated

In [1]:
import io
import os
import re
from pathlib import Path

import gspread
import numpy as np
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Google API

These initial steps will create a connection object to the [Google Drive API](https://developers.google.com/drive/api/v3/about-sdk) called `service`.  A valid authentication key from Google must be saved as key.json in the project's root directory.  This project uses [service accounts](https://cloud.google.com/iam/docs/understanding-service-accounts) for authentication to both the Google [Drive](https://developers.google.com/drive/api/v3/about-sdk) and [Sheets](https://developers.google.com/sheets/api/reference/rest) APIs.

In [2]:
CLIENT_SECRET_FILE = 'key.json'
SCOPES = ['https://www.googleapis.com/auth/drive', 
          'https://www.googleapis.com/auth/spreadsheets']

creds = service_account.Credentials.from_service_account_file(CLIENT_SECRET_FILE, scopes=SCOPES)

DATA_DIR = 'data'
DB_DIR = 'db'

DB_PATH = Path() / 'db'
DATA_PATH = Path() / 'data'

service = build('drive', 'v3', credentials=creds)

In [3]:
def create_service(api_key, api_name, api_version, scope):

    creds = service_account.Credentials.from_service_account_file(api_key, scopes=scope)

    try:
        service = build(api_name, api_version, credentials=creds)
        print(api_name, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None


# get_file_id

The purpose of this function is to ensure that we reference the correct folders by way of a uniquely assigned file ID value rather than the name of the folder, which may not always be unique within the project or even a folder. [Relevant Google Drive API Documentation](https://developers.google.com/drive/api/v3/reference/files)

The function will return the ID value of the specified `file_name` as a string object.  Optionally, a `parent_id` value can be specified in order to constrain results to subfolders found within a specific folder.  If multiple folders are found, it will return the first value and print a warning message.

In [4]:
def get_file_id(service, file_name, mime_type=None, parent_id=None):
    """Return the ID of a Google Drive file

    :param service: A Google Drive API service object
    :param file_name: A string, the name of the file
    :param mime_type: A string, optional MIME type of file to search for
    :param parent_id: A string, optional id of a parent folder to search in

    :return file_id: A string, file ID of the first found result
    """

    file_id = None

    query = """name='{}'
               and trashed=False
               """.format(file_name)

    if parent_id:
        query += "and parents in '{}'".format(parent_id)

    if mime_type:
        query += "and mimeType in '{}'".format(mime_type)

    try:
        results = service.files().list(
            q=query,
            fields='files(name, id)').execute()

        if len(results['files']) > 1:
            print('Multiple files found, retrieving first from list')

        file_id = results['files'][0]['id']

    except Exception as e:
        print('An error occurred: {}'.format(e))

    return file_id


Using `get_file_id`, we'll first assign the id of the root folder titled ***data*** into a variable called `data_folder_id`.  The ***data*** folder is where all of our unmodified raw data files will be placed.  We'll then search for the folder title `bls` within the data folder by passing `data_folder_id` as the `parent_id`.  

This function helps us minimize the risk of pulling files from the wrong folder.  It also helps to ensure that our project can be easily rebuilt from scratch (if necessary), because the id values are generated by Google Drive even if the file or folder names are the same as before.  This way, the user maintaining the system will not have to keep track of confusing id values in the event that folders or files have to be recreated--we'll just programmatically pull the right ones each time.

In [5]:
data_folder_id = get_file_id(service, 'data', 'application/vnd.google-apps.folder')
download_folder_id = get_file_id(service, 'bls_msa', 'application/vnd.google-apps.folder', parent_id=data_folder_id)

print(download_folder_id)

12Z6wn3e4lJKXl9Zcr9-3bYRsbTvLt7GW


# download_file

The purpose of this function is to download local copies of the raw data files from the remote Google Drive repository.  It is intended to help troubleshooting of unanticipated issues and errors during the ETL steps.  It will skip any files that are already saved locally.

In [6]:
def download_file(service, file_id, file_name, download_path):
    """Downloads a file from Google Drive to project folder

    :param service: A Google Drive API service object
    :param file_id: A string, file ID of the file to download
    :param file_name: A string, name and extension of file to save locally
    :param download_path: A pathlib.Path() object, folder to save file in

    :return None
    """

    file_path = download_path / file_name

    if file_path.is_file():
        print('...{} exists locally, skipping...'.format(file_path))
    else:
        request = service.files().get_media(fileId=file_id)

        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fd=fh, request=request)

        done = False

        while not done:
            status, done = downloader.next_chunk()
            print('...Downloading {}... {}'.format(file_path, status.progress() * 100))

        fh.seek(0)

        with open(file_path, 'wb') as f:
            f.write(fh.read())
            f.close()


Below, we'll use the id of the remote ***bls*** folder that we retrieved previously into `download_folder_id`, and use it to generate a list of id and filename pairs into a variable called `results`.

In [7]:
results = service.files().list(q="parents in '{}'".format(download_folder_id), fields='files(name, id)').execute()

results['files']

[{'id': '1chAdHqD_PFbH3pPDnuqukHW-Br22MN5o', 'name': 'MSA_M2014_dl.xlsx'},
 {'id': '1sRfrNCBvgyszPrymcUdJfATeZY1TUzsE', 'name': 'MSA_M2016_dl.xlsx'},
 {'id': '1xGRn9V1nYckEB02yxbPZhCAzQYJFf-Wl', 'name': 'MSA_M2015_dl.xlsx'},
 {'id': '1eyCpKwuZbw3vstD0F4Zwoc9chjKmPH8P', 'name': 'MSA_M2018_dl.xlsx'},
 {'id': '1pWjWI2em_LSm8LdmxgMIygSJ4m5WoH9e', 'name': 'MSA_M2020_dl.xlsx'},
 {'id': '1Ew1n5IHlf6B5sG2UNlOo0Zdr3ocj73Bu', 'name': 'MSA_M2017_dl.xlsx'},
 {'id': '14uMQPu6IneUp0k88vjsNoxzrJfB3sU8a', 'name': 'MSA_M2019_dl.xlsx'}]

We can then use `results` list to loop through and download all of the files locally.

In [8]:
for f in results['files']:
    download_file(service, f['id'], f['name'], DATA_PATH / 'bls_msa')

...data\bls_msa\MSA_M2014_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2016_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2015_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2018_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2020_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2017_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2019_dl.xlsx exists locally, skipping...


Lastly, we can put these pieces together to loop through and download all files from each of the three data folders.

In [9]:
""" Downloads all remote data files to local project folder"""

data_folder_id = get_file_id(service,
                             file_name=DATA_DIR,
                             mime_type='application/vnd.google-apps.folder')

sub_folders = ['bls_national', 'bls_msa', 'bg', 'ipeds', 'lookups']

print("Downloading remote files to local project...")
for folder in sub_folders:
    dl_folder_id = get_file_id(service, folder, parent_id=data_folder_id)
    results = service.files().list(
        q="parents in '{}' and trashed=False".format(dl_folder_id),
        fields='files(name, id)').execute()

    download_path = DATA_PATH / folder


    for file in results['files']:
        download_file(service, file['id'], file['name'], download_path)

...data\bls_national\national_M2018_dl.xlsx exists locally, skipping...
...data\bls_national\national_M2016_dl.xlsx exists locally, skipping...
...data\bls_national\national_M2019_dl.xlsx exists locally, skipping...
...data\bls_national\national_M2015_dl.xlsx exists locally, skipping...
...data\bls_national\national_M2014_dl.xlsx exists locally, skipping...
...data\bls_national\national_M2020_dl.xlsx exists locally, skipping...
...data\bls_national\national_M2017_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2014_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2016_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2015_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2018_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2020_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2017_dl.xlsx exists locally, skipping...
...data\bls_msa\MSA_M2019_dl.xlsx exists locally, skipping...
...data\bg\burning_glass_aggregate.xlsx exists locally, skippi

# load_msa_lookup / load_oes_lookup

Once the files are downloaded, we'll need to go through and process them.  One of the main data integration tasks in this project is to apply appropriate lookup crosswalks to each dataset.  Before we work on our main datasets, we'll first need to prepare our lookup tables.

In [10]:
def load_msa_lookup(data_path):
    """ Loads a lookup table for various MSA specific attributes used
    throughout the project.

    :param data_path: a pathlib.Path() object, location of data file folder

    :return msa_lookup: a dictionary of dictionaries containing MSA code to
        peer type lookup and MSA code to MSA name lookup

    """

    lookup_dir = 'lookups'
    msa_file_name = 'lk_msa.xlsx'

    lookup_file_path = data_path / lookup_dir / msa_file_name

    msa_lookup = {}

    try:
        lk_msa = pd.read_excel(lookup_file_path)
        msa_lookup['peer_type'] = dict(zip(lk_msa['area'], lk_msa['peer_type']))
        msa_lookup['area_title'] = dict(zip(lk_msa['area'], lk_msa['area_title']))
        print("...MSA lookup table loaded.")
        return msa_lookup
    except Exception as e:
        print("MSA lookup table loading failed - {}".format(e))
        return None

In [11]:
def load_oes_lookup(data_path):
    """ Loads a crosswalk for SOC codes provided by Bureau of Labor Statistics
    https://www.bls.gov/oes/soc_2018.htm

    :param data_path: A pathlib.Path() object, location of data file folder

    :return soc_lookup: A dictionary containing crosswalks for various years
    """

    lookup_dir = 'lookups'
    oes_file_name = 'oes_2019_hybrid_structure.xlsx'

    lookup_file_path = data_path / lookup_dir / oes_file_name

    try:
        # skip first 5 rows that contain notes
        df = pd.read_excel(lookup_file_path, sheet_name=0, skiprows=5)

        # rename column headers for easier manipulation
        df.columns = ['oes_code_2019', 'oes_title_2019',
                      'soc_code_2018', 'soc_title_2018',
                      'oes_code_2018', 'oes_title_2018',
                      'soc_code_2010', 'soc_title_2010', 'notes']

        # create SOC2010 to OES2019 for 2015 and 2016
        # create OES2018 to OES2019 for unmatched 2015 and 2016, and 2017-2020
        soc1019_dict = dict(zip(df['soc_code_2010'], df['oes_code_2019']))
        oes1819_dict = dict(zip(df['oes_code_2018'], df['oes_code_2019']))
        oes1919_dict = dict(zip(df['oes_code_2019'], df['oes_code_2019']))
        oes19_dict = dict(zip(df['oes_code_2019'], df['oes_title_2019']))

        soc_lookup = {'1019': soc1019_dict,
                      '1819': oes1819_dict,
                      '1919': oes1919_dict,
                      '19': oes19_dict}
        print("...SOC lookup tables loaded.")
        return soc_lookup
    except Exception as e:
        print("SOC lookup table loading failed - {}".format(e))
        return None

In [12]:
soc_lookup = load_oes_lookup(DATA_PATH)
msa_lookup = load_msa_lookup(DATA_PATH)

...SOC lookup tables loaded.
...MSA lookup table loaded.


# Process BLS files

The following functions perform various cleaning operations on each BLS data set.:
- `much_consistency()` - handles minor inconsistencies in raw data discovered through manual review
- `map_soc()` - depending on the report year, applies the correct crosswalks to standardize all occupations to the OES 2018 taxonomy
- `map_peer_type()` - labels peer MSAs for ease of aggregation in the analysis layer (National Peers, Alabama Peers, Birmingham)
- `map_nulls()` - Handles \*, \**, and \# values in the raw BLS data.  For now, we overwrite these values with nulls

In [13]:
def much_consistency(df):
    """ Helper function to handle known inconsistencies in raw data files

    :param df: A pandas dataframe, BLS OEWS data set

    :return df: A pandas dataframe
    """

    # Force lower case column headers
    df.columns = map(str.lower, df.columns)
    print("...Forced column headers to lower case.")

    # Rename column headers used inconsistently from year to year
    df.rename(columns={'occ_group': 'o_group',
                       'loc quotient': 'loc_quotient',
                       'area_name': 'area_title'}, inplace=True)
    print("...Standardized column header names.")

    return df

In [14]:
def map_soc(df, report_year, soc_lookup):
    """ Helper function to apply SOC code crosswalks for older BLS OEWS data.
    See: https://www.bls.gov/oes/soc_2018.htm

    :param df: A pandas dataframe, BLS OEWS data set
    :param report_year: An integer, release year of data set
    :param soc_lookup: A dictionary, crosswalk loaded via load_oes_lookup()

    :return df: A pandas dataframe
    """

    soc_mappings = soc_lookup['1919']

    if report_year in [2014, 2015, 2016]:
        soc_mappings = soc_lookup['1019']
    elif report_year in [2017, 2018]:
        soc_mappings = soc_lookup['1819']
    elif report_year >= 2019:
        soc_mappings = soc_lookup['1919']

    # Apply SOC code crosswalk
    df['oes_code_2019'] = df[df['o_group'] == 'detailed']['occ_code'].map(soc_mappings)

    # Map the couple codes that didn't have a 2010 to 2019 conversion separately
    df_matched = df[(~df['oes_code_2019'].isnull()) & (df['o_group'] == 'detailed')].copy()
    df_missing = df[(df['oes_code_2019'].isnull()) & (df['o_group'] == 'detailed')].copy()
    df_missing['oes_code_2019'] = df_missing['occ_code'].map(soc_lookup['1819'])

    # Map the total and major rows separately, because these aren't in the federal crosswalk
    df_totals = df[df['o_group'] != 'detailed'].copy()
    df_totals['oes_code_2019'] = df_totals['occ_code']
    df_totals['oes_title_2019'] = df_totals['occ_title']

    # Recombine all rows
    df_combined = pd.concat([df_matched, df_missing])
    df_combined['oes_title_2019'] = df_combined['oes_code_2019'].map(soc_lookup['19'])

    df_f = pd.concat([df_combined, df_totals])

    print("...Applied SOC crosswalks.")

    return df_f

In [15]:
def map_peer_type(df, msa_lookup):
    """ Helper function to apply peer group mapping to MSA locations

    :param df: A pandas dataframe, BLS OEWS data set
    :param msa_lookup: a dictionary containing MSA code to peer type lookup

    :return df: A pandas dataframe

    """

    # Map MSA areas to peer type categories
    df['peer_type'] = df['area'].map(msa_lookup['peer_type'])
    df['peer_type'].fillna("All Other MSA", inplace=True)
    print("...Applied MSA lookups.")

    return df

In [16]:
def map_msa_names(df, msa_lookup):
    """ Helper function to handle known MSA name changes/inconsistencies

    :param df: A pandas dataframe, BLS OEWS data set
    :param msa_lookup: a dictionary containing MSA code to peer type lookup

    :return df: A pandas dataframe
    """

    df['area_title'] = df['area'].map(msa_lookup['area_title'])

    return df

In [17]:
def map_nulls(df):
    """ Helper function to convert missing or unreported values into nulls

    :param df: A pandas dataframe, BLS OEWS data set

    :return df: a pandas dataframe
    """
    null_replace = {'#': np.nan, '*': np.nan, '**': np.nan}
    df.replace(null_replace, inplace=True)
    print("...Replaced missing and unreported values with nulls.")

    return df

`clean_bls_msa()` is the main cleaning function for the MSA specific files that loops through each raw data file and performs each of the individual cleaning steps.  Once all steps are performed, it will constrain the resulting dataset to just peer MSAs identified via `map_peer_types()`.

In [18]:
def clean_bls_msa(data_path):
    """

    :param data_path: A pathlib.Path() object, location of data file folder

    :return df_merged: A pandas dataframe, cleaned and merged dataframe
                       containing all available years of MSA BLS OEWS data
    """
    df_merged = pd.DataFrame()

    bls_folder = data_path / 'bls_msa'

    soc_lookup = load_oes_lookup(data_path)
    msa_lookup = load_msa_lookup(data_path)

    for f in bls_folder.glob('**/*.xlsx'):
        print('Processing {} ...'.format(f))

        with open(f, 'rb') as file:
            df = pd.read_excel(file)

        df = much_consistency(df)

        report_year = int(re.search(r"(M)([1-9]\d{3,})(_)", str(f))[2])

        df = map_soc(df, report_year, soc_lookup)
        df = map_peer_type(df, msa_lookup)

        df = map_nulls(df)
        df['report_year'] = report_year
        
        df['occ_code'] = df['oes_code_2019']
        df['occ_title'] = df['oes_title_2019']
        
        df['tech_occ'] = df['occ_code'].str.slice(0,3).map({'15-':'Tech'})
        df['tech_occ'].fillna("Non-Tech", inplace=True)

        df = df[['report_year', 'area', 'area_title', 'peer_type',
                 'occ_code', 'occ_title', 'tech_occ', 'o_group',
                 'tot_emp', 'emp_prse', 'jobs_1000', 'loc_quotient',
                 'h_mean', 'a_mean', 'mean_prse',
                 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
                 'a_pct10', 'a_pct25', 'a_median', 'a_pct75', 'a_pct90']
                ].reset_index(drop=True)

        df_merged = pd.concat([df_merged, df])
        print('...{} processing complete.'.format(f))

    df_merged = df_merged[df_merged['peer_type'] != 'All Other MSA']
    #df_merged = df_merged[df_merged['oes_code_2019'].str.contains('15-')]

    df_merged = map_msa_names(df_merged, msa_lookup)

    return df_merged

In [19]:
def clean_bls_national(data_path):
    """

    :param data_path: A pathlib.Path() object, location of data file folder

    :return df_merged: A pandas dataframe, cleaned and merged dataframe
                       containing all available years of national BLS OEWS data
    """
    df_merged = pd.DataFrame()

    bls_folder = data_path / 'bls_national'

    soc_lookup = load_oes_lookup(data_path)
    ###msa_lookup = load_msa_lookup(data_path)

    for f in bls_folder.glob('**/*.xlsx'):
        print('Processing {} ...'.format(f))

        with open(f, 'rb') as file:
            df = pd.read_excel(file)

        df = much_consistency(df)

        report_year = int(re.search(r"(M)([1-9]\d{3,})(_)", str(f))[2])

        df = map_soc(df, report_year, soc_lookup)
        ### df = map_peer_type(df, msa_lookup)

        df = map_nulls(df)
        df['report_year'] = report_year
        
        df['occ_code'] = df['oes_code_2019']
        df['occ_title'] = df['oes_title_2019']
        
        df['tech_occ'] = df['occ_code'].str.slice(0,3).map({'15-':'Tech'})
        df['tech_occ'].fillna("Non-Tech", inplace=True)

        df = df[['report_year', 'occ_code', 'occ_title', 'tech_occ', 'o_group',
                 'tot_emp', 'emp_prse', 'h_mean', 'a_mean', 'mean_prse',
                 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
                 'a_pct10', 'a_pct25', 'a_median', 'a_pct75', 'a_pct90']
                ].reset_index(drop=True)

        df_merged = pd.concat([df_merged, df])
        print('...{} processing complete.'.format(f))

    return df_merged

In [20]:
soc_lookup = load_oes_lookup(DATA_PATH)
msa_lookup = load_msa_lookup(DATA_PATH)

...SOC lookup tables loaded.
...MSA lookup table loaded.


The following command will run all cleaning steps and return a dataframe that contains all of o_group = total / major / detailed.

In [21]:
bls_merged_msa = clean_bls_msa(DATA_PATH)

...SOC lookup tables loaded.
...MSA lookup table loaded.
Processing data\bls_msa\MSA_M2014_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
...Applied SOC crosswalks.
...Applied MSA lookups.
...Replaced missing and unreported values with nulls.
...data\bls_msa\MSA_M2014_dl.xlsx processing complete.
Processing data\bls_msa\MSA_M2015_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
...Applied SOC crosswalks.
...Applied MSA lookups.
...Replaced missing and unreported values with nulls.
...data\bls_msa\MSA_M2015_dl.xlsx processing complete.
Processing data\bls_msa\MSA_M2016_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
...Applied SOC crosswalks.
...Applied MSA lookups.
...Replaced missing and unreported values with nulls.
...data\bls_msa\MSA_M2016_dl.xlsx processing complete.
Processing data\bls_msa\MSA_M2017_dl.xlsx ...
...Forced column headers to lower case.
...Stand

In [22]:
bls_merged_national = clean_bls_national(DATA_PATH)

...SOC lookup tables loaded.
Processing data\bls_national\national_M2014_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
...Applied SOC crosswalks.
...Replaced missing and unreported values with nulls.
...data\bls_national\national_M2014_dl.xlsx processing complete.
Processing data\bls_national\national_M2015_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
...Applied SOC crosswalks.
...Replaced missing and unreported values with nulls.
...data\bls_national\national_M2015_dl.xlsx processing complete.
Processing data\bls_national\national_M2016_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
...Applied SOC crosswalks.
...Replaced missing and unreported values with nulls.
...data\bls_national\national_M2016_dl.xlsx processing complete.
Processing data\bls_national\national_M2017_dl.xlsx ...
...Forced column headers to lower case.
...Standardized column header names.
.

### split_bls_ogroup

This function will take the resulting `bls_merged` dataset and split it into three separate files, each containing a slice of o_group = total / major / detailed.  *For this project's requirements,* separating these three files will make it easier to perform the majority of necessary calculations in the analysis and visualization layers.

In [23]:
def split_bls_ogroup(df):
    """ Helper function to split the BLS OEWS data into three separate
    dataframes each containing a different o_group level.

    :param df: A pandas dataframe, BLS OEWS data that has been cleaned

    :return bls_total: A pandas dataframe, o_group = total
    :return bls_major: A pandas dataframe, o_group = major
    :return bls_detailed: A pandas dataframe, o_group = detailed
    """
    bls_total = df[df['o_group'] == 'total']
    bls_major = df[df['o_group'] == 'major']
    bls_detailed = df[df['o_group'] == 'detailed']

    print("BLS data split into total, major, detailed.")
    return bls_total, bls_major, bls_detailed

In [24]:
df_total_msa, df_major_msa, df_detailed_msa = split_bls_ogroup(bls_merged_msa)

df_total_msa.to_excel(DB_PATH / 'bls_msa.total.xlsx', index=False)
df_major_msa.to_excel(DB_PATH / 'bls_msa.major.xlsx', index=False)
df_detailed_msa.to_excel(DB_PATH / 'bls_msa.detailed.xlsx', index=False)

BLS data split into total, major, detailed.


In [25]:
df_total_national, df_major_national, df_detailed_national = split_bls_ogroup(bls_merged_national)

df_total_national.to_excel(DB_PATH / 'bls_national.total.xlsx', index=False)
df_major_national.to_excel(DB_PATH / 'bls_national.major.xlsx', index=False)
df_detailed_national.to_excel(DB_PATH / 'bls_national.detailed.xlsx', index=False)

BLS data split into total, major, detailed.


# IPEDS

In [26]:
def ipeds_consistency(df):
    """ Helper function to handle known inconsistencies in raw data files

    :param df: A pandas dataframe, IPEDS Completions data set

    :return df: A pandas dataframe
    """
    
    # Strip whitespace from columns headers
    df.rename(columns=lambda x: x.strip(), inplace=True)

    # Force lower case column headers
    df.columns = map(str.lower, df.columns)
    print("...Forced column headers to lower case.")
    
    df['cipcode'] = df['cipcode'].apply(fix_cip_code)

    return df

In [27]:
import re
def fix_cip_code(cip):
    """
    Formats a cip code into the full 8 digit xx.xxxxxx format
    
    :param cip: A string, cip code
    
    :return: A string, 8 digit cip code in xx.xxxxxx format
    """
    cip = str(cip)
    if cip == '99':
        return '99'
    
    left, right = '', ''
    s = re.search(r"^(\d{1,2})(\.?)(\d{1,6})?$", cip)
    try:
        left = s[1].zfill(2)
        try:
            right = s[3].ljust(4, '0')
        except:
            right = ''.ljust(4, '0')
    except:
        #print('Invalid CIP code - {}'.format(str(cip)))
        return ""

    return ('.').join([left, right])

In [28]:
def simple_features(df):
    df['awards_total'] = df['ctotalt']
    df['awards_female'] = df['ctotalw']
    df['awards_urm_total'] = df['ctotalt'] - df['cwhitt']
    df['awards_urm_female'] = df['ctotalw'] - df['cwhitw']
    df['awards_blk_total'] = df['cbkaat']
    df['awards_blk_female'] = df['cbkaaw']
    
    df['awards_asian_total'] = df['casiat']
    df['awards_asian_female'] = df['casiaw']
    
    df['awards_amindian_total'] = df['caiant']
    df['awards_amindian_female'] = df['caianw']
    
    df['awards_hisp_total'] = df['chispt']
    df['awards_hisp_female'] = df['chispw']
    
    df['awards_hawaiian_total'] = df['cnhpit']
    df['awards_hawaiian_female'] = df['cnhpiw']
    
    df['awards_white_total'] = df['cwhitt']
    df['awards_white_female'] = df['cwhitw']
    
    df['awards_multi_total'] = df['c2mort']
    df['awards_multi_female'] = df['c2morw']
    
    df['awards_unknown_total'] = df['cunknt']
    df['awards_unknown_female'] = df['cunknw']
    
    df['awards_nralien_total'] = df['cnralt']
    df['awards_nralien_female'] = df['cnralw']
    
    return df
    

In [29]:
def load_ipeds_sector_lookup(data_path):
    """ Loads a description table IPEDS UNITID Sectors
    
    :param data_path: A pathlib.Path() object, location of data file folder

    :return ipeds_sector: A dictionary containing sector to title descriptions
    """
    lookup_dir = 'lookups'
    file_name = 'lk_sectors.xlsx'
    
    file_path = data_path / lookup_dir / file_name
    
    df = pd.read_excel(file_path, sheet_name=0)
    df.columns = map(str.lower, df.columns)
    
    lk_ipeds_sector = dict(zip(df['sector'], df['sector_title']))
    return lk_ipeds_sector

In [30]:
def load_awlevel_lookup(data_path):
    """ Loads a description table IPEDS awlevel
    
    :param data_path: A pathlib.Path() object, location of data file folder

    :return ipeds_sector: A dictionary containing awlevel to title descriptions
    """
    lookup_dir = 'lookups'
    file_name = 'lk_awlevel.xlsx'
    
    file_path = data_path / lookup_dir / file_name
    
    df = pd.read_excel(file_path, sheet_name=0)
    df.columns = map(str.lower, df.columns)
    
    lk_awlevel = dict(zip(df['awlevel'], df['award_title']))
    return lk_awlevel

In [31]:
def load_cip1020_lookup(data_path):
    """ Loads a crosswalk for CIP2010 to CIP2020
    https://nces.ed.gov/ipeds/cipcode/resources.aspx?y=56

    :param data_path: A pathlib.Path() object, location of data file folder

    :return cip1020_lookup: A dictionary containing crosswalks for various conversions
    """
    
    lk_cip1020 = {}
    
    lookup_dir = 'lookups'
    file_name = 'Crosswalk2010to2020.csv'
    
    file_path = data_path / lookup_dir / file_name
    
    df = pd.read_csv(file_path)
    df.columns = map(str.lower, df.columns)
    
    df['cipcode2010'] = df['cipcode2010'].str.replace("=", "").str.replace('"', "").apply(fix_cip_code)
    df['cipcode2020'] = df['cipcode2020'].str.replace("=", "").str.replace('"', "").apply(fix_cip_code)
    
    df = df[df['cipcode2010'] != ""].drop_duplicates()
    
    lk_cip1020['2010_to_2020'] = dict(zip(df['cipcode2010'], df['cipcode2020']))
    lk_cip1020['2020_titles'] = dict(zip(df['cipcode2020'], df['ciptitle2020']))
    
    return lk_cip1020

In [32]:
def load_tech_cip_lookup(data_path):

    """ Loads a crosswalk for CIP to SOC and SOC to CIP published by NCES
    https://nces.ed.gov/ipeds/cipcode/resources.aspx?y=56

    :param data_path: A pathlib.Path() object, location of data file folder

    :return cip_soc_lookup: A dictionary containing crosswalks for various conversions
    """

    lookup_dir = 'lookups'
    sipsoc_file_name = 'CIP2020_SOC2018_Crosswalk.xlsx'

    lookup_file_path = data_path / lookup_dir / sipsoc_file_name

    # CIP to SOC codes
    df = pd.read_excel(lookup_file_path, sheet_name='CIP-SOC')
    df.columns = map(str.lower, df.columns)
    df['cip2020code'] = df['cip2020code'].apply(fix_cip_code)

    tech_cips = list(df[df['soc2018code'].str[:2] == '15']['cip2020code'].unique())
    lk_tech_cips = dict(zip(tech_cips, ['Tech' for c in tech_cips]))

    print("...Tech CIP lookup table loaded.")
    return lk_tech_cips
    

In [33]:
import zipfile

def clean_ipeds(data_path):
    """

    :param data_path: A pathlib.Path() object, location of data file folder

    :return df_merged: A pandas dataframe, cleaned and merged dataframe
                       containing all available years of BLS OEWS data
    """
    df_c_merged = pd.DataFrame()

    ipeds_folder = data_path / 'ipeds'
    
    msa_lookup = load_msa_lookup(data_path)
    cip1020_lookup = load_cip1020_lookup(data_path)
    sector_lookup = load_ipeds_sector_lookup(data_path)
    tech_cip_lookup = load_tech_cip_lookup(data_path)
    awlevel_lookup = load_awlevel_lookup(data_path)

    r = r"(C)([1-9]\d{3,})(_A)"
    files = [f.name for f in os.scandir(ipeds_folder) if re.search(r, str(f.name))]
    
    for f in files:
        match = re.search(r, f)
        name = match[0]
        report_year = match[2]

        archive = zipfile.ZipFile(ipeds_folder / '{}.zip'.format(name))
        
        try:
            with archive.open('{}_rv.csv'.format(name.lower())) as csv:
                df_c = pd.read_csv(csv)
        except:
            with archive.open('{}.csv'.format(name.lower())) as csv:
                df_c = pd.read_csv(csv)
                
        hd_file = 'HD{}'.format(report_year)
        archive = zipfile.ZipFile(ipeds_folder / '{}.zip'.format(hd_file))
        try: 
            with archive.open('{}.csv'.format(hd_file.lower())) as csv:
                df_h = pd.read_csv(csv, encoding='latin1')
        except Exception as e:
            print('{} not found - {}'.format(hd_file, e))
            
        
        df_c = ipeds_consistency(df_c)
        df_c = simple_features(df_c)

        df_c = df_c[df_c['majornum']==1]
        
        df_h.columns = map(str.lower, df_h.columns)
        df_h = df_h[['unitid', 'instnm', 'cbsa', 'sector', 'longitud', 'latitude']]
        
        df_c['report_year'] = report_year
        df_c = pd.merge(df_c, df_h, on='unitid', how='left')
        df_c['area_title'] = df_c['cbsa'].map(msa_lookup['area_title'])
        df_c['peer_type'] = df_c['cbsa'].map(msa_lookup['peer_type'])

        df_c = df_c[df_c['cipcode'] != '99.0000']
        
        df_c['cipcode'] = df_c['cipcode'].map(cip1020_lookup['2010_to_2020'])
        df_c['cipcode_title'] = df_c['cipcode'].map(cip1020_lookup['2020_titles'])
        df_c['tech_cip'] = df_c['cipcode'].map(tech_cip_lookup)
        df_c['tech_cip'].fillna("Non-Tech", inplace=True)
        df_c['sector_title'] = df_c['sector'].map(sector_lookup)
        
        df_c['award_title'] = df_c['awlevel'].map(awlevel_lookup)
        
        df_c = df_c[['report_year', 'unitid', 'instnm', 'cbsa', 'area_title', 'peer_type', 'sector', 
                     'sector_title', 'longitud', 'latitude', 'cipcode', 'cipcode_title', 'tech_cip', 
                     'awlevel', 'award_title', 'awards_total', 'awards_female', 'awards_urm_total', 
                     'awards_urm_female', 'awards_blk_total', 'awards_blk_female', 'awards_asian_total',
                     'awards_asian_female', 'awards_amindian_total', 'awards_amindian_female',
                     'awards_hisp_total', 'awards_hisp_female', 'awards_hawaiian_total', 'awards_hawaiian_female',
                     'awards_white_total', 'awards_white_female', 'awards_multi_total', 'awards_multi_female',
                     'awards_unknown_total', 'awards_unknown_female', 'awards_nralien_total', 'awards_nralien_female']]
        
        df_c = df_c[~df_c['area_title'].isnull()]
        
        df_c_merged = pd.concat([df_c_merged, df_c])

    return df_c_merged

In [34]:
df_ipeds = clean_ipeds(DATA_PATH)

...MSA lookup table loaded.
...Tech CIP lookup table loaded.
...Forced column headers to lower case.
...Forced column headers to lower case.
...Forced column headers to lower case.
...Forced column headers to lower case.
...Forced column headers to lower case.
...Forced column headers to lower case.
...Forced column headers to lower case.


In [35]:
df_ipeds.shape

(53443, 37)

In [36]:
df_ipeds.to_excel(DB_PATH / 'ipeds.awards.xlsx', index=False)

# Awards by SOC Code

In [37]:
cipsoc = pd.read_excel(DATA_PATH / 'lookups' / 'CIP2020_SOC2018_Crosswalk.xlsx', sheet_name='CIP-SOC',dtype={'CIP2020Code': object})
lk_cipsoc = cipsoc.set_index('CIP2020Code')[['SOC2018Code', 'SOC2018Title']]
df_soc_awards = df_ipeds.join(lk_cipsoc, on='cipcode')
df_soc_awards.rename(columns={'SOC2018Code': 'occ_code', 'SOC2018Title': 'occ_title'}, inplace=True)
df_soc_awards['tech_occ'] = df_soc_awards['occ_code'].str.slice(0,3).map({'15-':'Tech'})
df_soc_awards['tech_occ'].fillna("Non-Tech", inplace=True)

In [38]:
# Tech occ codes only due to Google Sheets 50M cell limit
df_soc_awards[df_soc_awards['tech_occ']=='Tech'].to_excel(DB_PATH / 'ipeds.tech_soc_awards.xlsx', index=False)

# Load Google Sheets with final data

Run the `tech_talent_data_upload_google.ipynb` file for the final data loading step.
